In [29]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

[nltk_data] Downloading package stopwords to /Users/Pegah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import sys
# Ensure that your kernel is using Python3
assert sys.version_info.major == 3

In [35]:
# import the json
import json
with open("updated_data.json", "r") as f:
    women_summaries = json.load(f)
    
# PEGAH (From A5)
print("Each woman's entry is a dictionary with the following keys...")
print(women_summaries[0].keys())

deduped_women_summaries = {}
for woman in women_summaries:
    if woman['name'] not in deduped_women_summaries:
        deduped_women_summaries[woman['name']] = woman['summary']
    else:
        deduped_women_summaries[woman['name']] += "\n" + woman['summary']

Each woman's entry is a dictionary with the following keys...
dict_keys(['summary', 'name'])


# Dedup

In [61]:
deduped_women = [{'summary': summary, 'name': name} for (name, summary) in list(deduped_women_summaries.items())]

# Build Inverted Index

In [54]:
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize("a more general-purpose tokenizer")

def build_inverted_index(msgs):
    """ Builds an inverted index from the messages.
    
    Arguments
    =========
    
    msgs: list of dicts.
        Each message in this list already has a 'toks'
        field that contains the tokenized message.
    
    Returns
    =======
    
    index: dict
        For each term, the index contains a list of
        tuples (doc_id, count_of_term_in_doc):
        index[term] = [(d1, tf1), (d2, tf2), ...]
        
    Example
    =======
    
    >> test_idx = build_inverted_index([
    ...    {'toks': ['to', 'be', 'or', 'not', 'to', 'be']},
    ...    {'toks': ['do', 'be', 'do', 'be', 'do']}])
    
    >> test_idx['be']
    [(0, 2), (1, 2)]
    
    >> test_idx['not']
    [(0, 1)]
    
    """
    # YOUR CODE HERE
    
    c = Counter()
    index = defaultdict(list)
    # for word in list, c[word] += 1 ===> {word: count}
    msg_ind = 0 # message index
    for message in msgs:
        for token in tokenizer.tokenize(message['summary']):
            c[token] += 1 #counting number of times word is used
        for word in c:
            index[word].append((msg_ind, c[word]))
        c.clear()
        msg_ind += 1
    return index

In [66]:
inv_idx = build_inverted_index(deduped_women)
print(inv_idx)

defaultdict(<class 'list'>, {"Vava'u": [(708, 1)], 'Uganda': [(147, 1), (994, 1), (1018, 1), (1098, 2)], 'paddle': [(653, 1)], 'Built': [(521, 1), (929, 1)], 'luminosity': [(923, 1)], 'Memory': [(670, 1)], 'Bohannan': [(371, 2)], 'Southwestern': [(23, 3), (1054, 1)], 'nystatin.': [(1264, 1)], 'Formative.': [(597, 1)], 'taxonomy': [(1246, 1)], 'Interim': [(1266, 1)], 'Sharing': [(163, 1)], 'Softlab': [(841, 1)], 'polity': [(783, 1)], 'Joy': [(563, 1), (626, 1), (1218, 1)], 'libre': [(813, 4)], 'Workshop': [(216, 1), (735, 1)], 'Rabitat': [(582, 1)], 'recordings': [(109, 2), (159, 1), (523, 1), (571, 1), (729, 1), (1220, 1)], 'prehistory': [(38, 1), (998, 1), (1000, 1), (1015, 2), (1070, 1), (1189, 1)], '1993': [(154, 1), (161, 1), (198, 1), (234, 1), (369, 1), (461, 1), (523, 2), (579, 1), (595, 1), (672, 1), (738, 1), (870, 1), (880, 1), (885, 1), (937, 1), (1048, 1), (1081, 1), (1082, 1), (1091, 2), (1182, 1), (1260, 1), (1293, 1)], 'Ecuador': [(116, 1), (268, 1), (1260, 2)], 'Ages': 

### Assigning an index for each woman:

In [57]:
# # PEGAH
# # Here, we will assign an index for each woman. This index will help us access data in numpy matrices.
# # woman_to_index = movie_id_to_index
# woman_to_index = {woman:index for index, woman in enumerate([d["name"] for d in women_summaries])}


# # and because it might be useful...
# woman_name_to_index = {name:woman_to_index[name] for name in [d['name'] for d in women_summaries]}
# woman_index_to_name = {v:k for k,v in woman_name_to_index.items()}

# # print("The index of \"{}\" is {}".format(women_summaries[7]['name'], woman_name_to_index[women_summaries[7]['name']]))

# TF-IDF vectorizer

In [58]:
n_feats = 5000 # QUESTION: What does this do?
doc_by_vocab = np.empty([len(deduped_women), n_feats])

def build_vectorizer(max_features, stop_words, max_df=0.8, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object
    
    Params: {max_features: Integer,
             max_df: Float,
             min_df: Float,
             norm: String,
             stop_words: String}
    Returns: TfidfVectorizer
    """
    # YOUR CODE HERE
    return TfidfVectorizer(stop_words=stop_words, max_features=max_features, max_df=max_df, min_df=min_df, norm=norm)
    raise NotImplementedError()
    
tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([d['summary'] for d in women_summaries]).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

# Cosine similarity

In [59]:
def get_sim(wom1, wom2, input_doc_mat, woman_name_to_index):
    """Returns a float giving the cosine similarity of 
       the two movie transcripts.
    
    Params: {mov1: String,
             mov2: String,
             input_doc_mat: Numpy Array,
             movie_name_to_index: Dict}
    Returns: Float (Cosine similarity of the two movie transcripts.)
    """
    # YOUR CODE HERE
    
    a = input_doc_mat[woman_name_to_index[wom1]] #this gives us the [......] freq of all words
    b = input_doc_mat[woman_name_to_index[wom2]] 
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [60]:
"""Check that get_sim returns the correct output"""
print("Similarity: Mary Anning vs. Lilias Armstrong")
print("======")
test1 = get_sim('Lilias Armstrong', 'Mary Anning', doc_by_vocab, woman_name_to_index)
print(test1)
assert test1 < 1 and test1 > 0
print("")

# print("Similarity: Star Wars vs. Star Trek: Generations")
# print("======")
# test2 = get_sim('star wars', 'star trek: generations', doc_by_vocab, movie_name_to_index)
# print(test2)
# assert test2 < 0.25 and test2 > 0.20

Similarity: Mary Anning vs. Lilias Armstrong
0.14088923737130635



In [62]:
# def build_woman_sims_cos(woman_index_to_name,input_doc_mat,woman_name_to_index,input_get_sim_method):
#     """Returns a movie_sims matrix of size (num_movies,num_movies) where for (i,j):
#         [i,j] should be the cosine similarity between the movie with index i and the movie with index j
        
#     Note: What you set the diagonal to doesn't really matter, 
#     but you should set it to 1 to indicate that all movies are trivially perfectly similar to themselves.
    
#     Params: {n_mov: Integer,
#              movie_index_to_name: String,
#              input_doc_mat: Numpy Array,
#              movie_name_to_index: Dict,
#              input_get_sim_method: Function}
#     Returns: Numpy Array 
#     """
#     n_wom = len(women_index_to_name)
#     woman_sims = np.ones((n_wom, n_wom)) #instantiate array
#     for i in range(0, n_wom):
#         for j in range(0, n_wom):
#             woman_sims[i,j] = input_get_sim_method(woman_index_to_name[i], woman_index_to_name[j], input_doc_mat, woman_name_to_index)
#     return woman_sims
#     raise NotImplementedError()

In [63]:
# woman_sims_cos = build_woman_sims_cos(woman_index_to_name, doc_by_vocab, woman_name_to_index, get_sim)

In [78]:
def index_search(query, index):
    """ Search the collection of documents for the given query
    
    Arguments
    =========
    
    query: string,
        The query we are looking for.
    
    index: an inverted index as above
    
    idf: idf values precomputed as above
    
    doc_norms: document norms as computed above
    
    Returns
    =======
    
    results, list of tuples (score, doc_id)
        Sorted list of results such that the first element has
        the highest score, and `doc_id` points to the document
        with the highest score.
        
    """

    query_words = tokenizer.tokenize(query.lower())
    q = {} #vector of weights by term for query 
    results = defaultdict(int) # document_id : score
    q_norm_squared = 0
    all_words = index.keys()
    
    for q_word in query_words:
        ## make a vector of terms in the query and their weights
        if q_word in all_words:
            q[q_word] = query_words.count(q_word)
    
    for q_i in q:
        if q_i in index:
            for rel_doc in index[q_i]:
                results[rel_doc[0]] += q[q_i] * rel_doc[1]
        q_norm_squared += q[q_i]**2
    
    res = [(k, v) for k, v in results.items()]
    return sorted(res, key=lambda x: x[1], reverse=True)

In [84]:
print(index_search("mother", inv_idx))
print(deduped_women[908])

[(908, 4), (1024, 2), (76, 2), (1, 1), (1218, 1), (331, 1), (772, 1), (261, 1), (455, 1), (1032, 1), (523, 1), (964, 1), (654, 1), (915, 1), (21, 1), (1292, 1), (791, 1), (836, 1), (153, 1), (668, 1), (415, 1), (1092, 1), (225, 1), (391, 1), (300, 1), (749, 1), (1203, 1), (376, 1), (441, 1)]
{'summary': 'Stanley Ann Dunham (November 29, 1942 – November 7, 1995) was an American anthropologist who specialized in the economic anthropology and rural development of Indonesia. She was the mother of Barack Obama, the 44th President of the United States.\nDunham was known as Stanley Ann Dunham through high school, then as Ann Dunham, Ann Obama, Ann Soetoro, Ann Sutoro, and finally after her second divorce as Ann Dunham. Born in Wichita, Kansas, Dunham spent her childhood in California, Oklahoma, Texas and Kansas, her teenage years in Mercer Island, Washington, and most of her adult life in Hawaii and Indonesia.\nDunham studied at the East–West Center and at the University of Hawaii at Manoa in